In [1]:
from bias_bench.model import models

from importlib import reload
reload(models)

# model_name =  "meta-llama/Meta-Llama-3.1-70B-Instruct"
model_name = "meta-llama/Meta-Llama-3-8B"

generator = models.Private_Generator(model_name)


KeyboardInterrupt: 

In [ ]:
from bias_bench.model import models
from importlib import reload
reload(models)

In [1]:
from bias_bench.model import models

model_name_1 =  "meta-llama/Meta-Llama-3.1-70B-Instruct"
# model_name_1 = "meta-llama/Meta-Llama-3-8B"

generator_1 = models.Private_Generator(model_name_1)



/home/yandan/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/yandan/.cache/huggingface/token
Login successful


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [1]:
import json
import random

stereoset_path = "../data/stereoset/test.json"

with open(stereoset_path, 'r') as file:
    stereoset_data = json.load(file)

stereoset_data = stereoset_data['data']["intersentence"]
# if args.percentage != 100:
# sample_size = int(len(stereoset_data) * (1 / 100))
# stereoset_data = random.sample(stereoset_data, sample_size)

In [14]:
sample_size = int(len(stereoset_data) * (float(20) / 100))
sample_size

1274

In [25]:
import json
import os
dump_stereoset_path = "../data/stereoset/adapt_part"
if not os.path.exists(dump_stereoset_path):
    os.makedirs(dump_stereoset_path)

for i in range(5):
    dump_data = []
    dump_data = stereoset_data[0+(i*sample_size):sample_size+(i*sample_size)]  
    if i == 4:
        dump_data = stereoset_data[0+(i*sample_size):] 
    # print(dump_data)
    with open(f'{dump_stereoset_path}/test_part_{(i+1)}.json', 'w') as json_file:
        # Dump the data to the file
        json.dump(dump_data, json_file)
    

In [3]:
from typing import List, Dict 
from bias_bench.adaptation.prompt_context import get_stereo_context
import re


def get_inter_stereo_prompt(instances) -> List[str]:
    model_prompts = list()
    for instance in instances:
        bias_type = instance['bias_type']
        target = instance['target']
        model_prompts += [f"""

            Task: Adapt the StereoSet Bias dataset from its USA context to a Singaporean context.
            Objectives: Adapt wording to fit the Singaporean context (race, gender, religion, profession). Preserve dataset structure and labels. Ensure cultural relevance for Singapore's multicultural society.
            
            Instance Details: target: 
            Modify to a relevant group in Singapore if possible.
            bias_type: Keep strictly the same.
            Context: Adjust to reflect Singaporean situations.
            Sentences & Labels: Rewrite sentences (Sentence_1, Sentence_2, and Sentence_3) to fit the Singaporean context, maintaining correct labels (stereotype, anti-stereotype, unrelated), assign each label to each sentence once.
            If Adaptation is Difficult: Leave unchanged and state "Reason: [reason]" or "Reason: NA".
            
            Singapore Bias Profile: {get_stereo_context(bias_type)}

            Format your response as a JSON object with "Adapted":
            Target: {target}
            bias_type: {bias_type}
            Context: {instance['context']}
            Sentence_1: {instance['sentences'][0]['sentence']}
            Label_1: {instance['sentences'][0]['gold_label']}
            Sentence_2: {instance['sentences'][1]['sentence']}
            Label_2: {instance['sentences'][1]['gold_label']}
            Sentence_3: {instance['sentences'][2]['sentence']}
            Label_3: {instance['sentences'][2]['gold_label']}
            Adapted: JSON"""
        ]
        

    return model_prompts
            # {{\"explanation\": \"<brief_explanation_less_than_50_words>\", \"answer\": \"<answer_letter>\"}}\nContext: {instance['context']}\nA) {instance['sentences']['sentence'][0]}\nB) {instance['sentences']['sentence'][1]}\nC) {instance['sentences']['sentence'][2]}\n\nAnswer JSON:"]
            # {{\"target\": "[Adapted Target]",\n \"bias_type\": "[Bias Type]",\n "context": "[Adapted Context]", "sentences": [{"sentence": "[Adapted Sentence 1]", "label": "[Label 1]"}, {"sentence": "[Adapted Sentence 2]", "label": "[Label 2]"}, {"sentence": "[Adapted Sentence 3]", "label": "[Label 3]"}], "reason": "[Your reason or 'NA']"}}


In [38]:
from typing import List, Dict 
from bias_bench.adaptation.prompt_context import get_stereo_context
import re


def get_inter_stereo_prompt(instances) -> List[str]:
    model_prompts = list()
    for instance in instances:
        bias_type = instance['bias_type']
        target = instance['target']
        model_prompts += [f"""Please adapt the StereoSet Bias dataset from its Western-oriented context to a Singaporean context. Objectives: - Adapt wording to align with the Singaporean context, focusing on **race, gender, religion, and profession**. **Adaptation Guidelines**: - target: Modify to a relevant group in Singapore if applicable; this field is optional for adaptation. - bias_type: Keep unchanged. - context: Adjust to reflect situations common in Singapore. - sentences and labels: Rewrite each sentence to align with the Singaporean context, preserving its intended bias category (stereotype, anti-stereotype, unrelated). Ensure each label is correctly assigned, and each bias type is used only once. **Handling Difficult Adaptations**: - If adaptation is challenging, leave the instance unchanged and provide a reason (maximum 50 words) in the format: `"reason: [your reason]"`. If you helped adapt the sentences, state `"reason: NA"`. **Singapore Bias Profile**: - {{get_stereo_context{bias_type}}} **Adaptation Instructions**: Given the following JSON object from the StereoSet dataset, provide the adapted version in the same format, ensuring each sentence has its label correctly assigned and including the `"reason"` field: {{ "target": "{target}", "bias_type": "{bias_type}", "context": "{instance['context']}", "sentences": [ {{"sentence": "{instance['sentences'][0]['sentence']}", "label": "{instance['sentences'][0]['gold_label']}"}}, {{"sentence": "{instance['sentences'][1]['sentence']}", "label": "{instance['sentences'][1]['gold_label']}"}}, {{"sentence": "{instance['sentences'][2]['sentence']}", "label": "{instance['sentences'][2]['gold_label']}"}} ], "reason": "[Your reason or 'NA']"}} Provide the adapted version in the above JSON format after 'Your answer: '. Your answer: JSON""".strip()]

    return model_prompts


In [ ]:
from bias_bench.adaptation import prompt_strings
from importlib import reload
reload(prompt_strings)

batch_prompts = get_inter_stereo_prompt(stereoset_data)

i = 4

prompt = batch_prompts[i]
original_set = stereoset_data[i]
batch_response = generator_1.response_generator(prompt)
# key_result = prompt_strings.extract_sample_from_response("stereoset", batch_response[0])
print(batch_response)


In [ ]:
prompt

In [ ]:
result_json = re.sub(f'^.*?{re.escape("{")}', '{', batch_response[0])
result_json

In [ ]:
import json
json.loads(result_json)['sentences']

In [ ]:
stereoset_data[2]

In [ ]:
print(stereoset_data[1]['context'])
print(stereoset_data[1]['sentences'])
print(stereoset_data[1]['bias_type'])
print(stereoset_data[1]['target'])
print(stereoset_data[1]['sentences'][0]['sentence'])
print(stereoset_data[1]['sentences'][0]['gold_label'])
print(stereoset_data[1]['sentences'][1]['sentence'])
print(stereoset_data[1]['sentences'][1]['gold_label'])
print(stereoset_data[1]['sentences'][2]['sentence'])
print(stereoset_data[1]['sentences'][2]['gold_label'])


In [ ]:
for instance in stereoset_data['data']['intersentence']:
    print(instance)

In [15]:
from bias_bench.model import models
from bias_bench.adaptation import prompt_strings

batch_prompts = prompt_strings.get_intra_stereo_prompt(stereoset_data)
batch_response = generator.response_generator()
key_result = prompt_strings.extract_sample_from_response("stereoset", batch_response[0])

In [ ]:

key_result

In [ ]:
batch_prompts[1]

In [ ]:
batch_response

In [ ]:
stereoset_data

In [4]:
import json

dump_data = {"version": "1.0-test-sg-adapted", "data": {"intersentence": ""}} 

with open("../data/stereoset/adapted/adapted_full_intersentence_test.json", 'r') as file:
    data = json.load(file)

dump_data["data"]["intersentence"] = data

with open("../data/stereoset/adapted/test_intersentence_sg.json", 'w', encoding='utf8') as dump_file:
    json.dump(dump_data, dump_file)




In [8]:
import pandas as pd

with open("../data/stereoset/adapted/test_intersentence_sg.json", 'r') as file:

    inter = json.load(file)["data"]["intersentence"]

with open("../data/stereoset/test.json") as o_file:
    original = json.load(o_file)["data"]["intersentence"]

items = {"item_id":"",
         "target_adapted":"", "target_original": "",
         "context_adapted":"","context_original":"",
         "bias_adapted":"","bias_original":"",
         "sentence_1_adapted":"","sentence_1_original":"",
         "sentence_1_label_adapted":"","sentence_1_label_original":"",
         "sentence_2_adapted":"","sentence_2_original":"",
         "sentence_2_label_adapted":"", "sentence_2_label_original":"",
         "sentence_3_adapted":"","sentence_3_original":"",
         "sentence_3_label_adapted":"","sentence_3_label_original":"",
}

df = pd.DataFrame([items])




In [9]:
df

,item_id,target_adapted,target_original,context_adapted,context_original,bias_adapted,bias_original,sentence_1_adapted,sentence_1_original,sentence_1_label_adapted,sentence_1_label_original,sentence_2_adapted,sentence_2_original,sentence_2_label_adapted,sentence_2_label_original,sentence_3_adapted,sentence_3_original,sentence_3_label_adapted,sentence_3_label_original
0,,,,,,,,,,,,,,,,,,,


In [ ]:
for instance in inter:
    id_adapt = instance["id"]
    for o_instance in original:
        id_original = o_instance["id"]
        if id_original == id_adapt:
            assert o_instance["bias_type"] == instance["bias_type"]

            items["item_id"] = id_original

            items["target_original"] = o_instance['target']
            items["target_adapted"] = instance["target"]

            items['bias_adapted'] = instance["bias_type"]
            items['bias_original'] = o_instance['bias_type']

            if (o_instance['context'] == instance['context'] &
                 o_instance['sentences'][0]['sentence'] == instance['context'][0]['sentence'] &
                 o_instance['sentences'][1]['sentence'] == instance['context'][1]['sentence'] &
                 o_instance['sentences'][2]['sentence'] == instance['context'][2]['sentence']):



            items["sentence_1_adapted"] = 


